In [1]:
import matplotlib
import copy
import logging
#import torch

#from tqdm.autonotebook import tqdm

from data.data_loader import Dataset
from data.germeval2017 import germeval2017_dataset

from misc.preferences import PREFERENCES
#from misc.visualizer import *
from misc.run_configuration import get_default_params, randomize_params
from misc import utils

from optimizer import get_default_optimizer
from criterion import NllLoss, LossCombiner

from models.transformer.encoder import TransformerEncoder
from models.softmax_output import SoftmaxOutputLayerWithCommentWiseClass
from models.transformer_tagger import TransformerTagger
from models.jointAspectTagger import JointAspectTagger
from models.transformer.train import Trainer
import pprint

In [2]:
PREFERENCES.defaults(
    data_root='./data/germeval2017',
    data_train='train_v1.4.tsv',    
    data_validation='dev_v1.4.tsv',
    data_test='test_TIMESTAMP1.tsv',
    early_stopping='highest_5_F1'
)
def load(hp, logger):
    dataset = Dataset(
        'germeval',
        logger,
        hp,
        source_index=0,
        target_vocab_index=2,
        data_path=PREFERENCES.data_root,
        train_file=PREFERENCES.data_train,
        valid_file=PREFERENCES.data_validation,
        test_file=PREFERENCES.data_test,
        file_format='.tsv',
        init_token=None,
        eos_token=None
    )
    dataset.load_data(germeval2017_dataset, verbose=False)
    return dataset

In [3]:
def load_model(dataset, hp, experiment_name):
    loss = LossCombiner(4, dataset.class_weights, NllLoss)
    transformer = TransformerEncoder(dataset.source_embedding,
                                     hyperparameters=hp)
    model = JointAspectTagger(transformer, hp, 4, 20, dataset.target_names)
    optimizer = get_default_optimizer(model, hp)
    trainer = Trainer(
                        model,
                        loss,
                        optimizer,
                        hp,
                        dataset,
                        experiment_name,
                        enable_tensorboard=False,
                        verbose=False)
    return trainer

In [4]:
experiment_name = 'ConvHyperParameterSearch'
use_cuda = True

In [5]:
# get general logger just for search
experiment_name = utils.create_loggers(experiment_name=experiment_name)
logger = logging.getLogger(__name__)
logger.info('Run hyper parameter random grid search for experiment with name ' + experiment_name)

Log path is  /data/home/felix/ABSA-Transformer/logs/ConvHyperParameterSearch/20190224/3


In [6]:
utils.get_current_git_commit()
logger.info('Current commit: ' + utils.get_current_git_commit())

In [7]:
num_optim_iterations = 5
logger.info('num_optim_iterations: ' + str(num_optim_iterations))

In [8]:
random_grid_search_ranges = {
    'batch_size': (10, 20),
    'num_encoder_blocks': (2, 6),
    'pointwise_layer_size': (32, 4000),
    'clip_comments_to': (80, 120),
    'learning_rate': (0, 1e-2),
    #'learning_rate_factor': (1e-3, 4),
    #'learning_rate_warmup': (1000, 10000),
    #'optim_adam_beta1': (0.5, 0.99),
    #'optim_adam_beta2': (0.5, 0.99),
    'dropout_rate': (0, 0.8),
    'output_conv_num_filters': (5, 300),
    'output_conv_kernel_size': (1, 6),
    'transformer_config': {
        'transformer_heads': [1, 2, 3, 4, 5, 6, 10, 12, 15, 20]
    }
}
logger.info(pprint.pformat(random_grid_search_ranges, indent=2))

In [9]:
default_hp = get_default_params(use_cuda)
default_hp.num_epochs = 25
default_hp.seed = None
default_hp.output_layer_type = 'sum'

logger.info(default_hp)
print(default_hp)

+------------------------------------+
|          Hyperparameters           |
+-------------------------+----------+
|        Parameter        |  Value   |
+-------------------------+----------+
|        batch_size       |    12    |
|        model_size       |   300    |
|    learning_rate_type   |   noam   |
|      learning_rate      |    0     |
|   learning_rate_warmup  |   4800   |
|   learning_rate_factor  |    2     |
|     optim_adam_beta1    |   0.9    |
|     optim_adam_beta2    |   0.98   |
|      early_stopping     |    5     |
|         use_cuda        |   True   |
|       n_enc_blocks      |    3     |
|         n_heads         |    6     |
|           d_k           |    50    |
|           d_v           |    50    |
|       dropout_rate      |   0.1    |
|   pointwise_layer_size  |   2048   |
|    output_layer_type    |   sum    |
| output_conv_num_filters |   300    |
| output_conv_kernel_size |    5     |
|    output_conv_stride   |    1     |
|   output_conv_padding  

In [10]:
dataset_logger = logging.getLogger('data_loader')
best_f1 = 0.0
best_model = None
best_hp = None
best_iteration = -1
for optim_iteration in range(num_optim_iterations):
        
    print(f'\n\n+-------------------------------------------------+\nOptim Iteration: {optim_iteration}\n\n')
    logger.info(f'\n\n=================================\nOptim Iteration: {optim_iteration}\n=================================')
    
    # generate iteration hyper parameters
    hp = randomize_params(default_hp, random_grid_search_ranges)
        
    logger.info('New Params:')
    logger.info(hp)
    print(hp)
    
    logger.debug('Load dataset')
    dataset = load(hp, dataset_logger)
    logger.debug('dataset loaded')
    logger.debug('Load model')
    trainer = load_model(dataset, hp, experiment_name)
    logger.debug('model loaded')
    
    logger.debug('Begin training')
    model = None
    try:
        result = trainer.train(use_cuda=hp.use_cuda, perform_evaluation=False)
        model = result['model']
    except Exception as err:
        logger.exception("Could not complete iteration " + str(optim_iteration))
        print(f'Could not complete iteration {optim_iteration} because of {str(err)}')
        continue
        
    # perform evaluation and log results
    result = None
    try:
        result = trainer.perform_final_evaluation(use_test_set=False, verbose=False)
    except Exception as err:
        logger.exception("Could not complete iteration evaluation for it " + str(optim_iteration))
        print(f'Could not complete iteration {optim_iteration} evaluation because of {str(err)}')
        continue
    
    it_f1 = result[1][1]
    if best_f1 < it_f1:
        best_f1 = it_f1
        best_model = model
        best_hp = copy.copy(hp)
        best_iteration = optim_iteration
        print('+-------------------------------------------------+')
        print(f'Best Valid Result: {best_f1}')
        print('+-------------------------------------------------+')
    else:
        print(f'\nValid Result: {best_f1}\n')    
    print('\n\n###################################################\n')
    
print('Best iteration: ' + str(best_iteration))
print('Best f1: ' + str(best_f1))
print('Best HP:')
print(best_hp)



+-------------------------------------------------+
Optim Iteration: 0


+-------------------------------------------------+
|                 Hyperparameters                 |
+-------------------------+-----------------------+
|        Parameter        |         Value         |
+-------------------------+-----------------------+
|        batch_size       |           20          |
|        model_size       |          300          |
|    learning_rate_type   |          noam         |
|      learning_rate      | 0.0008692870511049277 |
|   learning_rate_warmup  |          4800         |
|   learning_rate_factor  |           2           |
|     optim_adam_beta1    |          0.9          |
|     optim_adam_beta2    |          0.98         |
|      early_stopping     |           5           |
|         use_cuda        |          True         |
|       n_enc_blocks      |           5           |
|         n_heads         |           10          |
|           d_k           |           30 

pre_training - DEBUG - 20 initialized
pre_training - DEBUG - Initilize parameters with nn.init.xavier_uniform_
pre_training - DEBUG - Tagger initialized


Epoch 1: 100%|██████████| 853/853 [02:42<00:00,  5.83it/s]

# EP	# IT	tr loss		val loss	f1		acc		duration / total time
1	853	186.32		170.61		0.204		0.757		2.71m - 2.7m / 0.0m


Epoch 2: 100%|██████████| 853/853 [05:18<00:00,  5.83it/s]

...

14	21700	2.55		13.18		0.303		0.913		1.90m - 26.6m / 47.5m


Epoch 16:   0%|          | 2/1550 [28:32<14:44:34, 34.29s/it]  

15	23250	2.39		12.60		0.321		0.931		1.90m - 28.5m / 47.6m


Epoch 16: 100%|██████████| 1550/1550 [30:26<00:00, 34.29s/it]  


16	24800	2.25		13.68		0.312		0.894		1.90m - 30.4m / 47.5m
+-------------------------------------------------+
Best Valid Result: 0.3137767038764019
+-------------------------------------------------+


###################################################

Best iteration: 4
Best f1: 0.3137767038764019
Best HP:
+------------------------------------------------+
|                Hyperparameters                 |
+-------------------------+----------------------+
|        Parameter        |        Value         |
+-------------------------+----------------------+
|        batch_size       |          11          |
|        model_size       |         300          |
|    learning_rate_type   |         noam         |
|      learning_rate      | 0.000738475015191512 |
|   learning_rate_warmup  |         4800         |
|   learning_rate_factor  |          2           |
|     optim_adam_beta1    |         0.9          |
|     optim_adam_beta2    |         0.98         |
|      early_stopping     |